# Imports

In [ ]:
# External
# from IPython.display import display
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# ibmpairs
import ibmpairs.authentication as authentication
import ibmpairs.catalog as catalog
import ibmpairs.client as client
import ibmpairs.query as query
import ibmpairs.dashboard as dashboard

# Authenticate & create Client

In [ ]:
#An auth/oauth2.txt file exists with content:
#<server>:<user>:<api_key>

# OAuth2
EIS_USER   = '<username>'
credentials  = authentication.OAuth2(username = EIS_USER)
eis_client = client.Client(authentication = credentials)

#A auth/basic.txt file exists with content:
#<server>:<user>:<password>

# or, Basic
#credentials = authentication.Basic(username = EIS_USER)
#eis_client = client.Client(authentication = credentials)

# Query

## Point

#### An example query to gather information about a spatial point.

In [ ]:
point_dict = {
  "layers": [
    {
      "id": "16100",
      "type": "raster"
    }
  ],
  "spatial": {
    "coordinates": [
      "35.7",
      "139.7"
    ],
    "type": "point"
  },
  "temporal": {
    "intervals": [
      {
        "end": "2018-10-31T00:00:00Z",
        "start": "2015-01-01T00:00:00Z"
      }
    ]
  }
}

In [ ]:
point_query = query.submit(point_dict)

In [ ]:
point_df = point_query.point_data_as_dataframe()

In [ ]:
dfpivot = point_df.pivot(index=['timestamp','latitude','longitude'], columns='layer_id',values=['value'])

In [ ]:
dfpivot

In [ ]:
# Analyse the returned data.

## Vector

#### An example vector query.

In [ ]:
vector_dict = {
  "layers": [
    {
      "id": "P265C4176",
      "type": "vector"
    }
  ],
  "outputType": "csv",
  "publish": False,
  "spatial": {
    "aoi": 24,
    "type": "poly"
  },
  "temporal": {
    "intervals": [
      {
        "end": "2018-11-14T00:00:00Z",
        "start": "2018-11-13T00:00:00Z"
      }
    ]
  }
}

In [ ]:
vector_query = query.submit_check_status_and_download(vector_dict)

In [ ]:
vector_files = vector_query.list_files()
print(vector_files)

In [ ]:
# Analyse the returned data.

## Raster

#### An example raster query.

In [ ]:
raster_dict = {
  "layers": [
    {
      "id": "51",
      "type": "raster"
    }
  ],
  "spatial": {
    "coordinates": [
      "35.6523",
      "-87.62",
      "51.6523",
      "-71.62"
    ],
    "type": "square"
  },
  "temporal": {
    "intervals": [
      {
        "snapshot": "2018-05-31T00:00:00Z"
      }
    ]
  }
}

In [ ]:
raster_query = query.submit_check_status_and_download(raster_dict)

In [ ]:
raster_files = raster_query.list_files()
print(raster_files)

In [ ]:
# Analyse the returned data.

## Favorite

#### Indicate to IBM Environmental Intelligence Suite: Geospatial that a query is a 'favorite'; this will prevent deletion by the cleaning process.

In [ ]:
raster_query.favorite()

## Unfavorite

#### Indicate to IBM Environmental Intelligence Suite: Geospatial that a query is no longer a 'favorite'.

In [ ]:
raster_query.unfavorite()

## Merge

#### In order to display and update IBM Environmental Intelligence Suite: Dashboard it is necessary to merge newly run queries in the same extent to a base query ID registered with the Dashboard component- this method perfoms this action.

In [ ]:
merge_dict = {
  "layers": [
    {
      "id": "49464",
      "type": "raster"
    }
  ],
  "name": "Area Query Test",
  "spatial": {
    "coordinates": [
      48.437249,
      2.5735152,
      48.488131,
      2.6838934
    ],
    "type": "square"
  },
  "temporal": {
    "intervals": [
      {
        "end": "2016-11-08",
        "start": "2016-11-06"
      }
    ]
  }
}

merge_query = query.query_from_dict(merge_dict)
merge_query.submit_and_check_status()

In [ ]:
other_merge_query = query.query_from_dict(merge_dict)
other_merge_query.temporal.intervals[0].start = "2018-02-04"
other_merge_query.temporal.intervals[0].end   = "2018-02-06"
other_merge_query.submit_and_check_status()

In [ ]:
merge_query.merge_query_into_base(other_job_id = other_merge_query)

In [ ]:
print(merge_query.merge_response)

## Replace Dates

#### The replace dates method replaces the all of the dates within a query object and sets a new nickname (optional), this is useful if a query object exists and the desire is to re-run with different date ranges.

In [ ]:
from datetime import datetime
start = datetime(2018,2,4,0,0,0)
end   = datetime(2018,2,6,0,0,0)

print(merge_query)

merge_query_with_new_dates = merge_query.replace_dates(start_date = start,
                                                       end_date   = end)

print("")
print(merge_query)

## Query by ID

#### Get previously run query by its ID number

In [ ]:
q_id = query.get_query_by_id(vector_query.id)
print(q_id)

## Latest Queries

#### Gather a list of recent queries made to IBM Environmental Intelligence Suite: Geospatial (default: 10), this method can be useful to retrive a query run on the UI.

In [ ]:
lq = query.get_latest_queries()
print(lq)

## Latest Favorites

#### Gather a list of recent queries, flagged as favorite, made to IBM Environmental Intelligence Suite: Geospatial (default: 10).

In [ ]:
lf = query.get_latest_favorites()
print(lf)

## Batch Query (Async)

#### Execute a number of queries asynchronously according to a number of async workers.

In [ ]:
raster_query_batch = query.query_from_dict(raster_dict)
vector_query_batch = query.query_from_dict(vector_dict)
queries = [raster_query_batch, vector_query_batch]

result = query.batch_query(queries = queries,
                           workers = 2,
                           verify = False)

print(result)

# Catalog

## Search

#### Search IBM Environmental Intelligence Suite: Geospatial catalog (Datasets, Datalayers) for an id or term.

In [ ]:
search_by_id = catalog.search("49464")
search_by_id

In [ ]:
search_by_word = catalog.search("Sentinel")
search_by_word

## Get a list of Datasets

#### Gather all Datasets a user has access to.

In [ ]:
ds_list = catalog.get_data_sets()
ds_list.display()

## Get a Dataset

#### Get a Dataset by an ID number 

In [ ]:
ds = catalog.get_data_set(id = "177")
ds.display()

## Get a list of Datalayers

#### Gather all Datalayers a user has access to.

In [ ]:
dl_list = catalog.get_data_layers()
dl_list.display()

## Get a list of Datalayers per Dataset

#### Gather all Datalayers in a given Dataset by ID number.

In [ ]:
dl_list_by_set = catalog.get_data_layers(data_set_id = "177")
dl_list_by_set.display()

## Get a Datalayer

#### Get a Datalayer by an ID number 

In [ ]:
dl = catalog.get_data_layer(id = "49464")
dl.display()

## Get a list of Datalayer Dimensions per Datalayer

#### Gather all Datalayer Dimensions in a given Datalayer by ID number.

In [ ]:
dlds = catalog.get_data_layer_dimensions(data_layer_id = "49166")
dlds.display()

## Get a Datalayer Dimension

#### Get a Datalayer Dimension by an ID number 

In [ ]:
dld = catalog.get_data_layer_dimension(id = "243")
dld.display()

# Dashboard

## Register Query with IBM Environmental Intelligence Suite: Dashboard

#### In order to display (and subsequently update) an IBM Environmental Intelligence Suite: Dashboard layer it is first necessary to register the query.

In [ ]:
area_query_dashboard_dict = {
  "debug": True,
  "layers": [
    {
      "id": "49464",
      "type": "raster"
    }
  ],
  "name": "Area Query Test",
  "spatial": {
    "coordinates": [
      48.437249,
      2.5735152,
      48.488131,
      2.6838934
    ],
    "type": "square"
  },
  "temporal": {
    "intervals": [
      {
        "end": "2016-11-08",
        "start": "2016-11-06"
      }
    ]
  }
}

In [ ]:
query_registration_result, query = dashboard.register_query(query      = area_query_dashboard_dict, 
                                                            query_name = 'examples_v2_query')
print('Query registration result = ' + str(query_registration_result))
print('Query result = ' + str(query.submit_response))
print('Query id = ' + str(query.id))

## Add Registered Query to IBM Environmental Intelligence Suite: Dashboard

#### In order to view the output of a registered query on the IBM Environmental Intelligence Suite: Dashboard component the add_dashboard_layer method can be used.

In [ ]:
query_registration_json = {"analyticsUuid": query_registration_result.analytics_uuid}
query_registration = dashboard.QueryRegistrationReturn.from_dict(query_registration_json)
dashboard.add_dashboard_layer(query_registration = query_registration, 
                              name               = "example_v2_layer")